In [21]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))
from features_glm import build_glm_features

df = pd.read_parquet("../data/work/freMTPL2_joined_capped.parquet")
df_glm = build_glm_features(df)

# sanity check
cols = ["VehAge","VehAgeGLM","DrivAge","DrivAgeGLM","VehPower","VehPowerGLM","BonusMalus","BonusMalusGLM","DensityGLM","Region"]
print(df_glm[cols].head(8))



out = "../data/work/freMTPL2_glm_ready.parquet"
df_glm.to_parquet(out, index=False)
print("Saved", out, "rows:", len(df_glm))


   VehAge VehAgeGLM  DrivAge DrivAgeGLM  VehPower VehPowerGLM  BonusMalus  \
0       0         1       55          6         5           5          50   
1       0         1       55          6         5           5          50   
2       2         2       52          6         6           6          50   
3       0         1       46          5         7           7          50   
4       0         1       46          5         7           7          50   
5       2         2       38          4         6           6          50   
6       2         2       38          4         6           6          50   
7       0         1       33          4         7           7          68   

   BonusMalusGLM  DensityGLM                Region  
0             50    7.104144           Rhone-Alpes  
1             50    7.104144           Rhone-Alpes  
2             50    3.988984              Picardie  
3             50    4.330733             Aquitaine  
4             50    4.330733             

In [22]:
# Laden
df_glm = pd.read_parquet("../data/work/freMTPL2_glm_ready.parquet").copy()

# Region als Kategorie 
df_glm["Region"] = df_glm["Region"].astype("category")

# Overall-Frequenz
overall_freq = df_glm["ClaimNb"].sum() / df_glm["Exposure"].sum()

# Kennzahlen je Region
reg = (df_glm.groupby("Region", observed=False)
       .agg(expo=("Exposure","sum"), claims=("ClaimNb","sum"))
       .assign(freq=lambda x: x["claims"] / x["expo"])
       .sort_values("expo", ascending=False))

# Zwei Kandidaten:
baseline_by_expo = reg.index[0]  # größte Exposure
baseline_by_overall = (reg.iloc[(reg["freq"] - overall_freq).abs().argmin()].name)

print("Overall freq:", round(overall_freq, 5))
print("Baseline (größte Exposure):", baseline_by_expo)
print("Baseline (freq ~ overall):", baseline_by_overall)


baseline_region = baseline_by_expo


#cats = [baseline_region] + [c for c in df_glm["Region"].cat.categories if c != baseline_region]
#df_glm["Region"] = df_glm["Region"].cat.reorder_categories(cats, ordered=True)

Overall freq: 0.10061
Baseline (größte Exposure): Centre
Baseline (freq ~ overall): Franche-Comte


In [28]:
from metrics import pdw, pdw2, cf2, make_folds

import pandas as pd, numpy as np

df = pd.read_parquet("../data/work/freMTPL2_glm_ready.parquet").copy()
df["fold"] = make_folds(len(df), k=5, seed=42)
learn, test = df[df["fold"]!=5], df[df["fold"]==5]

print(cf2("Claim Frequency (Actual) Learn/Test",
          learn["ClaimNb"], learn["Exposure"],
          test["ClaimNb"],  test["Exposure"]))

cf = learn["ClaimNb"].sum() / learn["Exposure"].sum()
learn_pred = cf * learn["Exposure"]
test_pred  = cf * test["Exposure"]

print(pdw2("Poisson Deviance INT",
           learn_pred, learn["ClaimNb"], learn["Exposure"],
           test_pred,  test["ClaimNb"],  test["Exposure"]))


Claim Frequency (Actual) Learn/Test: 10.07% / 10.03%
Poisson Deviance INT, Learn/Test: 36.92% / 36.90%
